In [ ]:
import numpy as np
import pandas as pd
import os
import sys
from PIL import Image
import matplotlib.pyplot as plt
import librosa
import json

In [ ]:
def create_dataset(folder_path): 
  images = []
  names = []
  for file in os.listdir(folder_path):
    image_path = os.path.join(folder_path, file)

    size = Image.open(image_path).size
    image = np.array(Image.open(image_path).resize((int(size[0]/4),int(size[1]/2))))
    image = image.astype('float32')
    image = np.delete(image, [3], axis=2)

    image /= 255.0
    if file[:-4] != 'axqtExFY5-s (1)':
      images.append(image)
      names.append(file[:-4])

  max_len = 0
  for i in range(len(images)):
    length = images[i].shape[1]
    if length>max_len:
      max_len=length

  for i in range(len(images)):
    images[i] = librosa.util.fix_length(images[i], int(max_len), axis=1)

  images = np.array(images)

  df = pd.read_csv('/content/drive/MyDrive/YouTubeID_and_Keywords.csv')

  keywords = []
  for i in names:
    if i in df.YTID.to_list():
      keys = df.Keywords[df.YTID == i].to_list()
      keywords.append(keys[0].split(';'))
    else:
      print(i)

  return names,images,keywords

In [ ]:
names, images, keywords = create_dataset('/content/drive/MyDrive/Pre-Training Images')

In [ ]:
all_words = []
for i in keywords:
  for j in i:
    if len(j)>2:
      all_words.append(j)

len(all_words)

45288

In [ ]:
from collections import Counter

frequency_list = Counter(all_words).most_common(300)

In [ ]:
keywords_updated = []
for j in range(len(keywords)):
  keyword_list = []
  for i in frequency_list:
    if i[0] in keywords[j]:
      keyword_list.append(i[0])
  keywords_updated.append(keyword_list)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
multihot = mlb.fit_transform(keywords_updated)
class_labels = list(mlb.classes_)
print(len(class_labels))

300


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, multihot, test_size=0.20, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(15100, 64, 107, 3) (3776, 64, 107, 3) (15100, 300) (3776, 300)


In [ ]:
from sklearn.utils import shuffle
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.optimizers import SGD
from keras.layers.pooling import AveragePooling2D
from keras.layers.normalization.batch_normalization import BatchNormalization
import tensorflow as tf

In [ ]:
# Create CNN model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(64, 107, 3)))
model.add(tf.keras.layers.Conv2D(64, 3, strides=2, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(128, 3, strides=2, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(512, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(512, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(1,1)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(512, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(1,1)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1024, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(len(class_labels), activation='softmax'))

In [ ]:
# Compile model
model.compile(
    loss='categorical_crossentropy',
    optimizer=SGD(learning_rate=0.0001, momentum=0.6),
    metrics=['accuracy'],
)

In [ ]:
history = model.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test))

Epoch 1/15
472/472 [==============================] - 338s 714ms/step - loss: 10.8471 - accuracy: 0.1026 - val_loss: 13.8781 - val_accuracy: 0.0143
Epoch 2/15
472/472 [==============================] - 336s 712ms/step - loss: 9.7396 - accuracy: 0.1750 - val_loss: 9.1714 - val_accuracy: 0.1992
Epoch 3/15
472/472 [==============================] - 339s 718ms/step - loss: 9.1554 - accuracy: 0.1961 - val_loss: 9.1774 - val_accuracy: 0.1796
Epoch 4/15
472/472 [==============================] - 333s 706ms/step - loss: 8.8469 - accuracy: 0.2043 - val_loss: 9.3853 - val_accuracy: 0.1721
Epoch 5/15
472/472 [==============================] - 345s 730ms/step - loss: 8.5767 - accuracy: 0.2080 - val_loss: 8.5161 - val_accuracy: 0.2079
Epoch 6/15
472/472 [==============================] - 344s 729ms/step - loss: 8.3783 - accuracy: 0.2126 - val_loss: 9.2501 - val_accuracy: 0.2177
Epoch 7/15
472/472 [==============================] - 344s 728ms/step - loss: 8.2327 - accuracy: 0.2167 - val_loss: 8.2767

In [ ]:
def create_testset(folder_path): 
  images = []
  names = []
  for file in os.listdir(folder_path):
    file = os.listdir(folder_path)[j]
    
    image_path = os.path.join(folder_path, file)

    size = Image.open(image_path).size
    image = np.array(Image.open(image_path).resize((int(size[0]/4),int(size[1]/2))))
    image = image.astype('float32')
    image = np.delete(image, [3], axis=2)

    image /= 255.0
    if file[:-4] != 'axqtExFY5-s (1)':
      images.append(image)
      names.append(file[:-4])

  for i in range(len(images)):
    images[i] = librosa.util.fix_length(images[i], 107, axis=1)

  images = np.array(images)

  return names,images

In [ ]:
test_names, test_images = create_testset('/content/drive/MyDrive/Test Images')

In [ ]:
predictions = model.predict(test_images)

In [ ]:
pred_labels = []
for j in predictions:
  pred_label = []
  for i in range(len(j)):
    sorted_labels = []
    pred_label.append(str(j[i]) + ' ' + class_labels[i])
    sorted_label = np.sort(pred_label)[::-1][:1]
    for i in sorted_label:
      a = i.split(' ')[1:]
      sorted_labels.append(' '.join(a))
  pred_labels.append(sorted_labels)

In [ ]:
!pip install panns-inference
!pip install keytotext --upgrade

In [ ]:
from keytotext import pipeline
nlp = pipeline("mrm8488/t5-base-finetuned-common_gen")
k2t_new = pipeline("k2t-new")

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/clotho_captions_evaluation.csv')
df.head(3)

,file_name,caption_1,caption_2,caption_3,caption_4,caption_5
0,Santa Motor.wav,A machine whines and squeals while rhythmicall...,A person is using electric clippers to trim bu...,Someone is trimming the bushes with electric c...,The whirring of a pump fills a bladder that tu...,"While rhythmically punching or stamping, a mac..."
1,Radio Garble.wav,A radio dispatcher and an officer are communic...,Communication with a walkie-talkie with a lot ...,A discussion with a walkie-talkie with a consi...,People talking through a walkie-talkie with ba...,The walkie-talkie the people are talking throu...
2,Radio Fuzz for Old Radio Broadcast FF233.wav,A radio tuner has been positioned in between r...,A transistor radio is being played on a statio...,A transistor radio is on a station that is not...,Radio static makes a constant hum with a high ...,Static coming from a radio that is in between ...


In [ ]:
candidates1 = []
candidates2 = []
references = []

for i in range(len(test_names)):
  sentence1 = k2t_new(pred_labels[i])
  candidates1.append(str(sentence1).split())
  sentence2 = nlp(pred_labels[i])
  candidates2.append(str(sentence2).split())
  reference = []
  test_sample = df[df.file_name == test_names[i]]
  reference.append(str(test_sample.caption_1.to_list()[0]).split())
  reference.append(str(test_sample.caption_2.to_list()[0]).split())
  reference.append(str(test_sample.caption_3.to_list()[0]).split())
  reference.append(str(test_sample.caption_4.to_list()[0]).split())
  reference.append(str(test_sample.caption_5.to_list()[0]).split())
  references.append(reference)

In [ ]:
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

In [ ]:
bl1 = corpus_bleu(references, candidates1, weights=(1, 0, 0, 0))
bl2 = corpus_bleu(references, candidates1, weights=(0, 1, 0, 0))

In [ ]:
print("1-gram bleu score:", bl1)
print("2-gram bleu score:", bl2)

1-gram bleu score: 0.27072868750467616
2-gram bleu score: 0.019185535765210256
